In [ ]:
import pandas as pd
import numpy as np

class CompanyLoanAnalyzer:
    def __init__(self):
        # Define risk thresholds and scoring weights
        self.risk_weights = {
            'current_ratio': 0.2,
            'debt_equity_ratio': 0.2,
            'interest_coverage': 0.15,
            'profit_margin': 0.15,
            'revenue_growth': 0.15,
            'cash_flow_coverage': 0.15
        }

        # Define baseline interest rates
        self.base_rate = 0.05  # 5% base rate
        self.risk_premium_max = 0.10  # Maximum 10% risk premium

    def calculate_financial_ratios(self, financial_data):
        """Calculate key financial ratios from company data."""
        ratios = {}

        # Current Ratio = Current Assets / Current Liabilities
        ratios['current_ratio'] = financial_data['current_assets'] / financial_data['current_liabilities']

        # Debt to Equity Ratio = Total Debt / Total Equity
        ratios['debt_equity_ratio'] = financial_data['total_debt'] / financial_data['total_equity']

        # Interest Coverage Ratio = EBIT / Interest Expenses
        ratios['interest_coverage'] = financial_data['ebit'] / financial_data['interest_expense']

        # Profit Margin = Net Income / Revenue
        ratios['profit_margin'] = financial_data['net_income'] / financial_data['revenue']

        # Revenue Growth = (Current Revenue - Previous Revenue) / Previous Revenue
        ratios['revenue_growth'] = (financial_data['revenue'] - financial_data['previous_revenue']) / financial_data['previous_revenue']

        # Cash Flow Coverage = Operating Cash Flow / Total Debt
        ratios['cash_flow_coverage'] = financial_data['operating_cash_flow'] / financial_data['total_debt']

        return ratios

    def calculate_risk_score(self, ratios):
        """Calculate overall risk score based on financial ratios."""
        risk_scores = {
            'current_ratio': min(1, ratios['current_ratio'] / 2),  # Target >= 2
            'debt_equity_ratio': max(0, 1 - ratios['debt_equity_ratio'] / 2),  # Target <= 1
            'interest_coverage': min(1, ratios['interest_coverage'] / 3),  # Target >= 3
            'profit_margin': min(1, max(0, ratios['profit_margin'] / 0.15)),  # Target >= 15%
            'revenue_growth': min(1, max(0, ratios['revenue_growth'] / 0.1)),  # Target >= 10%
            'cash_flow_coverage': min(1, ratios['cash_flow_coverage'] / 0.3)  # Target >= 30%
        }

        # Calculate weighted average risk score
        total_score = sum(score * self.risk_weights[metric]
                         for metric, score in risk_scores.items())

        return total_score

    def determine_loan_amount(self, financial_data, risk_score):
        """Determine recommended loan amount based on financials and risk score."""
        # Calculate maximum loan amount based on various factors
        max_amounts = [
            financial_data['revenue'] * 0.5,  # Up to 50% of revenue
            financial_data['operating_cash_flow'] * 3,  # Up to 3x operating cash flow
            (financial_data['total_assets'] - financial_data['total_debt']) * 0.7  # Up to 70% of net assets
        ]

        # Base maximum amount on the most conservative metric
        base_max_amount = min(max_amounts)

        # Adjust based on risk score (0.5 to 1.0 multiplier)
        recommended_amount = base_max_amount * (0.5 + 0.5 * risk_score)

        return recommended_amount

    def calculate_interest_rate(self, risk_score):
        """Calculate recommended interest rate based on risk score."""
        # Higher risk score = lower risk premium
        risk_premium = self.risk_premium_max * (1 - risk_score)
        return self.base_rate + risk_premium

    def analyze_company(self, financial_data):
        """Main analysis function to provide loan recommendations."""
        # Calculate financial ratios
        ratios = self.calculate_financial_ratios(financial_data)

        # Calculate risk score
        risk_score = self.calculate_risk_score(ratios)

        # Determine recommended loan amount
        recommended_amount = self.determine_loan_amount(financial_data, risk_score)

        # Calculate interest rate
        interest_rate = self.calculate_interest_rate(risk_score)

        return {
            'risk_score': risk_score,
            'recommended_loan_amount': recommended_amount,
            'interest_rate': interest_rate,
            'financial_ratios': ratios
        }

def get_financial_data():
    """Get financial data from user input."""
    print("\nPlease enter the following financial data (in millions):")

    financial_data = {}
    fields = [
        ('current_assets', 'Current Assets'),
        ('current_liabilities', 'Current Liabilities'),
        ('total_debt', 'Total Debt'),
        ('total_equity', 'Total Equity'),
        ('ebit', 'EBIT (Earnings Before Interest & Tax)'),
        ('interest_expense', 'Interest Expense'),
        ('net_income', 'Net Income'),
        ('revenue', 'Current Revenue'),
        ('previous_revenue', 'Previous Year Revenue'),
        ('operating_cash_flow', 'Operating Cash Flow'),
        ('total_assets', 'Total Assets')
    ]

    for field_key, field_name in fields:
        while True:
            try:
                value = float(input(f"Enter {field_name}: "))
                if value < 0:
                    print("Please enter a non-negative value.")
                    continue
                financial_data[field_key] = value
                break
            except ValueError:
                print("Please enter a valid number.")

    return financial_data

def main():
    print("Welcome to the Company Loan Analyzer")
    print("====================================")

    # Get financial data from user
    financial_data = get_financial_data()

    # Create analyzer instance
    analyzer = CompanyLoanAnalyzer()

    # Get recommendations
    result = analyzer.analyze_company(financial_data)

    # Print results
    print("\nAnalysis Results")
    print("===============")
    print(f"Risk Score: {result['risk_score']:.2f}")
    print(f"Recommended Loan Amount: ${result['recommended_loan_amount']:,.2f}")
    print(f"Recommended Interest Rate: {result['interest_rate']:.1%}")
    print("\nFinancial Ratios:")
    for ratio, value in result['financial_ratios'].items():
        print(f"{ratio}: {value:.2f}")

if __name__ == "__main__":
    main()

Welcome to the Company Loan Analyzer

Please enter the following financial data (in millions):
